# Model Autoscaling 

Let's use asyncio to send a bunch of requests at our URL.
**Important**: First, enter the inference endpoint URL you got after deploying the model server.

In [ ]:
# CHANGE the value below if it says userX, or nothing will work!
RestURL = 'https://my-first-model-userX.apps.cluster-x4rsj.x4rsj.sandbox2796.opentlc.com'

In [ ]:
# If you did not use the Workbench image designed for this Lab, you can uncomment and run the following line to install the required packages.
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

In [ ]:
import asyncio
import requests
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

# Import your custom blocking function
from remote_infer import process_image 

# Tell matplotlib to display plots inline in the notebook
%matplotlib inline

print("Libraries imported.")

## Here is our asyncio function to loop over our 5 images.

In [ ]:
async def run_concurrent_inference():
    """
    An ASYNCHRONOUS function that runs multiple blocking
    functions concurrently in a thread pool.
    """
    # Get the currently running asyncio event loop (Jupyter has one)
    loop = asyncio.get_running_loop()
    
    # --- Setup your inference calls ---
    infer_url = f'{RestURL}/v2/models/my-first-model/infer' # Using your URL
    
    # Updated to use your 5 specific images
    image_paths = [
        'images/carImage0.jpg',
        'images/carImage1.jpg',
        'images/carImage2.jpg',
        'images/carImage3.jpg',
        'images/carImage4.jpg',
    ]
    # Number of times we will send all of our image paths
    repetitions = 5

    print(f"Starting {len(image_paths) * repetitions} concurrent inference calls...")
    
    # Create a list of tasks to run.
    # loop.run_in_executor(None, ...) runs the blocking function
    # (process_image) in a separate thread pool.
    tasks = []
    for i in range(0,repetitions):
        for path in image_paths:
            # We pass the imported 'process_image' function here
            task = loop.run_in_executor(None, process_image, path, infer_url)
            tasks.append(task)
            sleep(1)
        
    # asyncio.gather waits for all tasks to complete.
    # The results will be a list of the images returned by process_image
    results = await asyncio.gather(*tasks)
    
    print("All inference calls have finished.")
    return results

## Now we call the function!

In [ ]:
# Make sure your 'RestURL' variable is defined before this cell
start_time = time.time()

# This is how you run it in Jupyter!
# Just 'await' the async function.
processed_images = await run_concurrent_inference()

end_time = time.time()

print(f"\n--- Total time taken: {end_time - start_time:.2f} seconds ---")

## Show the result

In [ ]:
print(f"Received {len(processed_images)} processed images.")

print("Printing first 5 processed images.")

# Display the results
fig, axes = plt.subplots(1, 5, figsize=(20, 5)) # Made figure wider
fig.suptitle("Concurrent Inference Results")

for i, img_bgr in enumerate(processed_images):
    if img_bgr is not None:
        if i < 5:
            # Convert from BGR (OpenCV) to RGB (Matplotlib)
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            axes[i].imshow(img_rgb)
            axes[i].set_title(f"Image {i+1}")
            axes[i].axis('off')
    else:
        axes[i].set_title(f"Image {i+1} (Failed)")
        axes[i].axis('off')

plt.show()